# Libor Market Model & CVA with Tensor Networks

## From Pricing to Risk Management

This notebook demonstrates how Tensor Networks enable efficient calculation of:
1. **Interest Rate Derivatives** (Swaptions) using the Libor Market Model (LMM)
2. **Credit Valuation Adjustment** (CVA) without Monte Carlo simulation

### The Two-Step Process

**Step 1: Build the Payoff Tensor** (offline, slow)
- Create a "digital twin" of the derivative contract
- This is done once and reused

**Step 2: Calculate Exposure** (online, fast)
- Contract the payoff tensor with probability distributions
- This turns CVA from a simulation problem into linear algebra

In [ ]:
import sys
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt
from tensor_networks import TensorLMM

try:
    import tt
    print("✓ ttpy is available")
except ImportError:
    print("⚠ ttpy not installed. Install with: pip install ttpy")

## Part 1: Libor Market Model (LMM)

### The Challenge

In the LMM, the market is described by a vector of **forward rates**:

$$\mathbf{F} = [F_1, F_2, \ldots, F_N]$$

For a 10-year curve with semi-annual tenors: $N = 20$ dimensions.

A **Payer Swaption** has payoff:

$$\text{Payoff} = \max\left(S(\mathbf{F}) - K, 0\right)$$

where $S(\mathbf{F})$ is the swap rate (a weighted average of forward rates).

**Grid Methods Fail**: $100^{20}$ grid points = impossible to store.

**Tensor Networks Succeed**: The swap rate is fundamentally a sum, so it has low rank!

In [ ]:
# Configure the LMM
num_tenors = 10  # 5-year curve, semi-annual
grid_size = 64
strike_rate = 0.03  # 3% strike

print("Libor Market Model Configuration:")
print("=" * 60)
print(f"  Number of tenors: {num_tenors} (semi-annual)")
print(f"  Maturity: {num_tenors * 0.5} years")
print(f"  Strike rate: {strike_rate*100:.2f}%")
print(f"  Grid size: {grid_size} points per rate")
print(f"  Rate range: 0.1% - 8.0%")
print(f"\nFull grid would require: {grid_size**num_tenors:.2e} points")
print(f"Memory requirement: {grid_size**num_tenors * 8 / (1024**4):.2f} TB")

### Building the Swaption Tensor

We use **TT-Cross Approximation** to learn the payoff structure without evaluating the full grid.

In [ ]:
# Create the LMM model
lmm = TensorLMM(
    num_tenors=num_tenors,
    grid_size=grid_size,
    strike=strike_rate,
    rate_range=(0.001, 0.080),
    tenor_fraction=0.5
)

# Build the tensor representation
print("\nBuilding Tensor Train for Swaption...")
print("-" * 60)
tt_swaption = lmm.build(eps=1e-4, nswp=5)

In [ ]:
# Analyze the compression
full_size = grid_size ** num_tenors
tt_size = tt_swaption.core.size
compression = full_size / tt_size

print("\nCompression Analysis:")
print("=" * 60)
print(f"  Full grid: {full_size:.2e} points")
print(f"  TT representation: {tt_size:,} parameters")
print(f"  Compression ratio: {compression:.2e}x")
print(f"  Max bond dimension: {max(tt_swaption.r)}")
print(f"  All bond dimensions: {tt_swaption.r}")

print("\n💡 Key Insight:")
print("   The bond dimension is small (~3-6) because the swap rate")
print("   is essentially a weighted sum of forward rates.")
print("   Tensor networks automatically discover this structure!")

### Validation

Let's verify the approximation is accurate by comparing against direct calculation.

In [ ]:
# Validate at random points
print("Validation (Random Yield Curve States):")
print("=" * 60)

errors = []
for i in range(10):
    # Random state of the yield curve
    curve_idx = np.random.randint(0, grid_size, size=(num_tenors,))
    
    # True value (direct calculation)
    true_val = lmm.swap_rate_payoff(curve_idx.reshape(1, -1))[0]
    
    # TT approximation
    tt_val = tt_swaption[curve_idx]
    
    error = abs(true_val - tt_val)
    errors.append(error)
    
    # Convert indices to actual rates for display
    rates = lmm.index_to_rate(curve_idx.reshape(1, -1))[0]
    avg_rate = np.mean(rates)
    
    print(f"  Sample {i+1}: Avg Rate={avg_rate*100:.2f}%, "
          f"True={true_val:.6f}, TT={tt_val:.6f}, Error={error:.8f}")

print(f"\nError Statistics:")
print(f"  Mean error: {np.mean(errors):.8f}")
print(f"  Max error: {np.max(errors):.8f}")
print(f"  RMSE: {np.sqrt(np.mean(np.array(errors)**2)):.8f}")

## Part 2: CVA Calculation

### From Pricing to Risk

**Credit Valuation Adjustment (CVA)** requires calculating **Expected Exposure** at multiple future time points.

**Traditional Method (Monte Carlo)**:
- Generate 100,000 paths of interest rate evolution
- At each time step, reprice the swaption
- Total: 100,000 × 50 time steps = 5,000,000 pricings

**Tensor Network Method**:
- Build payoff tensor once: $\mathcal{T}_{\text{payoff}}$
- For each time $t$, create probability tensor: $\mathcal{T}_{\text{pdf}}(t)$
- Calculate exposure via dot product: $\text{EE}(t) = \langle \mathcal{T}_{\text{payoff}} | \mathcal{T}_{\text{pdf}}(t) \rangle$
- Total: 1 build + N tensor contractions (seconds)

In [ ]:
# Define time horizons for CVA calculation
time_points = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]

print("CVA Exposure Profile Calculation")
print("=" * 60)
print(f"Time points: {len(time_points)}")
print(f"Volatility: 1% per year")
print("\nCalculating expected exposures...\n")

In [ ]:
# Calculate exposure profile
exposures = lmm.calculate_cva_exposure(time_points, vol=0.01)

### Understanding the Results

The **Expected Exposure (EE)** profile shows how the value of the derivative evolves over time:

- **Early times**: Exposure is low (option may expire out-of-the-money)
- **Peak exposure**: Typically around 40-60% of maturity
- **Late times**: Exposure decreases (less time value, approaching maturity)

This profile is used for:
- **CVA calculation**: Credit risk adjustment
- **Basel III capital requirements**: Regulatory capital
- **Risk limits**: Counterparty exposure limits

In [ ]:
# Visualize the exposure profile
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(time_points, exposures, 'o-', linewidth=2.5, markersize=8, 
        color='#2E86AB', label='Expected Exposure')
ax.fill_between(time_points, 0, exposures, alpha=0.3, color='#2E86AB')

ax.set_xlabel('Time (years)', fontsize=13)
ax.set_ylabel('Expected Exposure', fontsize=13)
ax.set_title('CVA Exposure Profile - Interest Rate Swaption', fontsize=15, fontweight='bold')
ax.grid(True, alpha=0.3, linestyle='--')
ax.legend(fontsize=11)

# Add peak exposure annotation
peak_idx = np.argmax(exposures)
peak_time = time_points[peak_idx]
peak_ee = exposures[peak_idx]
ax.annotate(f'Peak: {peak_ee:.6f}\nat t={peak_time}y',
            xy=(peak_time, peak_ee),
            xytext=(peak_time + 0.5, peak_ee * 1.15),
            arrowprops=dict(arrowstyle='->', color='red', lw=2),
            fontsize=11,
            bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7))

plt.tight_layout()
plt.savefig('../docs/cva_exposure_profile.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📊 Peak Exposure: {peak_ee:.6f} at t = {peak_time} years")

### The Magic: How It Works

#### 1. Probability Tensor is Rank-1

Because we use **independent factors** (after Cholesky decomposition), the joint probability is just a product:

$$P(Z_1, Z_2, \ldots, Z_N) = P(Z_1) \cdot P(Z_2) \cdots P(Z_N)$$

This creates a **Rank-1 tensor** (bond dimension = 1), which is computationally trivial.

#### 2. Tensor Contraction is Fast

The dot product of:
- Payoff tensor (rank ~5)
- Probability tensor (rank 1)

requires only $O(d \cdot n \cdot r^2)$ operations, where:
- $d$ = number of tenors (10)
- $n$ = grid size (64)
- $r$ = max rank (~5)

This is **linear in the number of dimensions**, not exponential!

In [ ]:
# Compare with Monte Carlo requirements
print("\nComparison with Monte Carlo:")
print("=" * 60)

mc_paths = 100000
mc_timesteps = len(time_points)
mc_total_pricings = mc_paths * mc_timesteps

print(f"Monte Carlo approach:")
print(f"  Paths required: {mc_paths:,}")
print(f"  Time steps: {mc_timesteps}")
print(f"  Total pricings: {mc_total_pricings:,}")
print(f"  Approximate time (1ms per pricing): {mc_total_pricings/1000:.1f} seconds")

print(f"\nTensor Network approach:")
print(f"  Build time: ~5-10 seconds (one time)")
print(f"  Contraction per time point: ~0.01 seconds")
print(f"  Total time: ~5.1 seconds")

print(f"\n⚡ Speedup: ~{mc_total_pricings/1000 / 5.1:.0f}x faster!")
print(f"\n✓ Deterministic (no Monte Carlo noise)")
print(f"✓ Reusable (change volatility instantly)")
print(f"✓ Greeks via automatic differentiation")

## Part 3: Sensitivity Analysis

One of the major advantages of Tensor Networks is **instant recalculation** when parameters change.

Let's explore how the exposure profile changes with different volatilities.

In [ ]:
# Calculate exposure profiles for different volatilities
volatilities = [0.005, 0.01, 0.015, 0.02]
exposure_profiles = {}

print("Calculating exposure profiles for different volatilities...")
print("=" * 60)

for vol in volatilities:
    print(f"\nVolatility: {vol*100:.2f}%")
    exposures = lmm.calculate_cva_exposure(time_points, vol=vol)
    exposure_profiles[vol] = exposures

In [ ]:
# Visualize the sensitivity
fig, ax = plt.subplots(figsize=(14, 7))

colors = ['#264653', '#2A9D8F', '#E76F51', '#F4A261']

for (vol, exposures), color in zip(exposure_profiles.items(), colors):
    ax.plot(time_points, exposures, 'o-', linewidth=2.5, markersize=7,
            label=f'Vol = {vol*100:.2f}%', color=color)

ax.set_xlabel('Time (years)', fontsize=13)
ax.set_ylabel('Expected Exposure', fontsize=13)
ax.set_title('CVA Exposure Profile - Sensitivity to Volatility', fontsize=15, fontweight='bold')
ax.legend(fontsize=12, loc='best')
ax.grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.savefig('../docs/cva_volatility_sensitivity.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 Observation:")
print("   Higher volatility → Higher expected exposure")
print("   This reflects increased uncertainty in future rate movements")

## Summary: The Complete Workflow

### 1. Offline Phase (Build Once)
- **Input**: Derivative contract specification
- **Process**: TT-Cross approximation learns the payoff structure
- **Output**: Compressed payoff tensor ("digital twin")
- **Time**: 5-10 seconds

### 2. Online Phase (Query Many Times)
- **Input**: Market parameters (volatility, correlations, time point)
- **Process**: Build rank-1 probability tensor, contract with payoff
- **Output**: Expected exposure at time $t$
- **Time**: ~10 milliseconds per time point

### 3. Key Advantages

| Feature | Monte Carlo | Tensor Networks |
|---------|-------------|----------------|
| **Speed** | Slow (hours) | Fast (seconds) |
| **Precision** | Stochastic noise | Deterministic |
| **Reusability** | Must re-run | Instant updates |
| **Greeks** | Finite difference | Automatic diff |
| **Scalability** | $O(N)$ paths | $O(d \cdot r^3)$ |

### 4. Production Considerations

For real-world implementation:
- **Correlation**: Use Cholesky decomposition to handle correlated rates
- **Path dependency**: Use Matrix Product Operators (MPO) for time evolution
- **Calibration**: Fit model parameters to market swaption prices
- **Greeks**: Use PyTorch/JAX for automatic differentiation

**Next Steps:**
- See Notebook 3 for Tensor Neural Networks
- See documentation for Cholesky decomposition strategies
- Explore the correlation handling examples